# Quality Control in Manufacturing - Automatic Defect Detection
<h3><span style="color: #117d30;"> Using Azure Custom Vision Congnitive Services</span></h3>

## Scenario Overview

**Azure Custom Vision** is a cognitive service which helps in building and deploying object detection and image classification models. 

This notebook provides an end-to-end demo on how to use the Azure Cognitive Services Custom Vision API to build and deploy a classification model for **automated sub-assembly inspection for quality control purpose**. 

In this scenario we will see how the model will classify whether a given item is good or defective. If the model classifies the given piece as defective, it will be automatically marked as defective.

### Setting-Up: 

+ Initiating Spark Context 
+ Defining the storage, file path and file name for **raw input** images
+ Defining the storage, file path and file name for **processed output** images
+ Downloading the images from an **ADLS Gen2 storage container**

In [3]:
%%pyspark

# importing libraries for ADLS Gen2 connection
import os
from pyspark.sql import SparkSession
from azure.storage.blob import BlockBlobService

# creating spark session 
session = SparkSession.builder.getOrCreate()

# blob configs
STORAGE_ACCOUNT_NAME = "#DATA_LAKE_NAME#"
STORAGE_ACCOUNT_ACCESS_KEY = "#STORAGE_ACCOUNT_KEY#"
STORAGE_CONTAINER_NAME = "qualitycontrol"

session.conf. \
 set("fs.azure.account.key.{}.blob.core.windows.net".format(STORAGE_ACCOUNT_NAME), \
 STORAGE_ACCOUNT_ACCESS_KEY)


# creating blob service object and list blobs inside input_folder
input_folder="input/"
output_folder="output/"
blob_service = BlockBlobService(STORAGE_ACCOUNT_NAME, STORAGE_ACCOUNT_ACCESS_KEY)
generator= blob_service.list_blobs(STORAGE_CONTAINER_NAME, prefix=input_folder)

# initializing several lists that will be used in the following cells
input_list=[]
output_list=[]
blob_file_list=[]
blob_output_list=[]

# creating the input and output folders if they do not exist
if not os.path.isdir("input"):
   os.mkdir("input")
if not os.path.isdir("output"):
   os.mkdir("output")

# getting the names of blob files and appending them to the list
for c in generator:
    blob_file_list.append(c.name)

# downloading the images from ADLS Gen2 storage
for i,blob_file in enumerate(blob_file_list):
     local_path = os.path.join(os.getcwd(), blob_file)
     blob_service.get_blob_to_path(STORAGE_CONTAINER_NAME, blob_file ,local_path)

# creating several lists
for i,image in enumerate(os.listdir("input")):
    full_path=os.path.join(os.getcwd(),"input",image)
    output_path=os.path.join(os.getcwd(),"output",image)
    blob_output="output/" + image
    input_list.append(full_path)
    output_list.append(output_path)
    blob_output_list.append(blob_output)

## Installing OpenCV and importing other python libraries  

OpenCV is one of the most popular computer vision libraries for image and video processing. In this notebook, we will install python binding for  OpenCV. 

We will also import other libraries like  **matplotlib**,  **numpy**, **pandas**, etc. that are required at different stages in this notebook for manipulating and processing images.    

In [4]:
%%pyspark

# installing OpenCv and grequests libraries and importing the OpenCV library 
# openCV will be used to convert the frames into arrays   
import pip
pip.main(["install","opencv-python-headless"])
pip.main(["install","grequests"])
import cv2 

# for processing arrays   
import numpy as np

# for encoding and decoding Custom Vision predictions 
import json

# for converting the Custom Vision predictions to dataframe   
import pandas as pd

# for making web service calls in batches
import grequests 

# for displaying images from the processed images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# importing other required libraries
import io
import time
from PIL import ImageFont, ImageDraw, Image

You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

### Displaying raw images before processing


In [5]:
%%pyspark

#displaying the raw images before processing
for _,file in enumerate(input_list):
   img = mpimg.imread(file)
   imgplot = plt.imshow(img)
   plt.show()

## Making API Call to Custom Vision Cognitive Service  

Now we will define connections details for making the API call to the Custom Vision Service 


In [6]:
%%pyspark 

# web service end-point foe the Custom Vision model    
# we will process images  
POST_URL = "https://westus2.api.cognitive.microsoft.com/customvision/v3.0/Prediction/e555e244-ebfd-481b-ae0e-f3213d8e5634/classify/iterations/Iteration1/image"

# providing prediction key
HEADERS = {'Prediction-Key': "7f743d4b8d6d459fb2bb0e8648dfa38e","Content-Type":"application/json"}

# number of API calls per pool of request   
MAX_CONNECTIONS = 100

## Functions to Converting Raw Input Images to Processed Images


### function: getFrames

This function takes a  list of image files as input and returns list of frames    


In [7]:
%%pyspark

# returns list of arrays from list of images
def getFrames(input_list):
    images = []
    byteImages = []
    for _,k in enumerate(input_list):
        image = cv2.imread(k)
        images.append(image)
        global HEIGHT, WIDTH
        HEIGHT, WIDTH, lay = images[0].shape
    return images

### function: convertCVImagesToByteImages

This function takes a list of frames as input and returns list of Byte arrays 

In [8]:
%%pyspark

# returns list of byte arrays
def convertCVImagesToByteImages(frames):
    toReturnByteImages = []
    for frame in frames:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        is_success, im_buf_arr = cv2.imencode(".jpg", image)
        if is_success:
            byte_im = im_buf_arr.tobytes()
            toReturnByteImages.append(byte_im)
    return toReturnByteImages

### function:  batchRequest 

This function takes a list of byteImages and returns the list of raw responses


In [9]:
%%pyspark

# returns list of raw responses
def batchRequest(byteImages):
    results = []
    for x in range(0,len(byteImages)+1, MAX_CONNECTIONS):
        rs = (grequests.post(POST_URL,headers=HEADERS,data=u,stream=False) for u in byteImages[x:x+MAX_CONNECTIONS])
        time.sleep(0.2)
        results.extend(grequests.map(rs)) 
    response_list=[response.text for response in results]
    return response_list

### function: getDFfromResponse

This function takes a JSON as input and convert them into dataframe.


In [10]:
%%pyspark

# converts JSON to Pandas dataframe 
def getDFfromResponse(response):
    resp = json.loads(response)
    pred_resp= resp['predictions']
    pred_df=pd.DataFrame(pred_resp)
    final_df = pred_df.head(1)
    return final_df

### function: get_label

This function takes predicted label as input and returns the text and color for that label


In [11]:
%%pyspark

# returns color and text for given prediction
def get_label(value):
    if value=="def_front":
        fill=(255,0,0,255)
        text="Defective"
    else:
        fill=(0,255,0,255)
        text="Good"
    return text,fill

### function: process_image

This function converts list of raw images into list of processed images. This function relies on several other functions that are defines above. 


In [12]:
%%pyspark 

# converts list of raw images to processed images
def process_image(input_list,output_list):
    frames = getFrames(input_list)
    byteImages=convertCVImagesToByteImages(frames)
    response_list=batchRequest(byteImages)
    fontPath = "/usr/share/fonts/truetype/dejavu/DejaVuSerif-Bold.ttf"
    sans16  =  ImageFont.truetype ( fontPath, 20 )
    
    for i,response in enumerate(response_list):
        df=getDFfromResponse(response)
        tag_value=(df['tagName'].head(1).values)[0]
        text,fill=get_label(tag_value)
        
        # Convert to PIL Image
        cv2_im_rgb = cv2.cvtColor(frames[i], cv2.COLOR_BGR2RGB)
        pil_im = Image.fromarray(cv2_im_rgb)

        draw = ImageDraw.Draw(pil_im)

        # Draw the text
        draw.text((0, 0), text,fill=fill,font=sans16 )

        # Save the image
        cv2_im_pcd = cv2.cvtColor(np.array(pil_im), cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_list[i], cv2_im_pcd)

## Image Conversion and ADLSGen2 Storage  

We wil make functions call to convert list of raw images to list of processed images and store them to ADLSGen2 storage. 


In [13]:
%%pyspark 
# converts list of raw images to list of processed images
process_image(input_list,output_list)

In [14]:
%%pyspark 
%%capture

# uploads processed images to an ADLS Gen2 Storage container
for i,blob_out in  enumerate(blob_output_list):
    blob_service.create_blob_from_path(
                 STORAGE_CONTAINER_NAME, blob_out, output_list[i])

### Displaying Processed images


In [15]:
%%pyspark

# displays the final processed images
for _,file in enumerate(output_list):
   img = mpimg.imread(file)
   imgplot = plt.imshow(img)
   plt.show()